In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "WeatherPy_Database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mokhsogollokh,RU,61.4681,128.9203,-26.86,100,84,3.00,broken clouds
1,1,Tarime,TZ,-1.3500,34.3667,72.32,51,51,4.88,broken clouds
2,2,Benguela,AO,-12.5763,13.4055,77.34,74,93,5.44,overcast clouds
3,3,Kruisfontein,ZA,-34.0033,24.7314,68.88,80,0,7.40,clear sky
4,4,Rikitea,PF,-23.1203,-134.9692,77.38,88,100,19.93,moderate rain


In [3]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Tarime,TZ,-1.3500,34.3667,72.32,51,51,4.88,broken clouds
2,2,Benguela,AO,-12.5763,13.4055,77.34,74,93,5.44,overcast clouds
3,3,Kruisfontein,ZA,-34.0033,24.7314,68.88,80,0,7.40,clear sky
4,4,Rikitea,PF,-23.1203,-134.9692,77.38,88,100,19.93,moderate rain
7,7,Vaini,TO,-21.2000,-175.2000,75.36,64,20,8.05,few clouds
14,14,Mar Del Plata,AR,-38.0023,-57.5575,81.82,78,75,6.91,broken clouds
15,15,Cape Town,ZA,-33.9258,18.4232,70.38,75,20,11.50,few clouds
16,16,Belmonte,BR,-15.8631,-38.8828,84.20,61,62,7.70,broken clouds
18,18,Kununurra,AU,-15.7667,128.7333,84.18,89,100,5.75,overcast clouds
25,25,Saint-Pierre,RE,-21.3393,55.4781,78.48,69,0,8.05,clear sky


In [5]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng
1,Tarime,TZ,72.32,broken clouds,-1.3500,34.3667
2,Benguela,AO,77.34,overcast clouds,-12.5763,13.4055
3,Kruisfontein,ZA,68.88,clear sky,-34.0033,24.7314
4,Rikitea,PF,77.38,moderate rain,-23.1203,-134.9692
7,Vaini,TO,75.36,few clouds,-21.2000,-175.2000
14,Mar Del Plata,AR,81.82,broken clouds,-38.0023,-57.5575
15,Cape Town,ZA,70.38,few clouds,-33.9258,18.4232
16,Belmonte,BR,84.20,broken clouds,-15.8631,-38.8828
18,Kununurra,AU,84.18,overcast clouds,-15.7667,128.7333
25,Saint-Pierre,RE,78.48,clear sky,-21.3393,55.4781


In [7]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Tarime,TZ,72.32,broken clouds,-1.3500,34.3667,
2,Benguela,AO,77.34,overcast clouds,-12.5763,13.4055,
3,Kruisfontein,ZA,68.88,clear sky,-34.0033,24.7314,
4,Rikitea,PF,77.38,moderate rain,-23.1203,-134.9692,
7,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,
14,Mar Del Plata,AR,81.82,broken clouds,-38.0023,-57.5575,
15,Cape Town,ZA,70.38,few clouds,-33.9258,18.4232,
16,Belmonte,BR,84.20,broken clouds,-15.8631,-38.8828,
18,Kununurra,AU,84.18,overcast clouds,-15.7667,128.7333,
25,Saint-Pierre,RE,78.48,clear sky,-21.3393,55.4781,


In [8]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}        

In [9]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Tarime,TZ,72.32,broken clouds,-1.3500,34.3667,
2,Benguela,AO,77.34,overcast clouds,-12.5763,13.4055,
3,Kruisfontein,ZA,68.88,clear sky,-34.0033,24.7314,
4,Rikitea,PF,77.38,moderate rain,-23.1203,-134.9692,
7,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,


In [13]:
# Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.replace('', np.nan)

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Tarime,TZ,72.32,broken clouds,-1.3500,34.3667,NaN
2,Benguela,AO,77.34,overcast clouds,-12.5763,13.4055,NaN
3,Kruisfontein,ZA,68.88,clear sky,-34.0033,24.7314,NaN
4,Rikitea,PF,77.38,moderate rain,-23.1203,-134.9692,NaN
7,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,NaN
